In [1]:
import numpy as np
import pandas as pd

In [2]:
with open('./eval.txt', 'r') as f:
    results = f.readlines()

# Remove any non-result lines from the eval file, and split the lines on the tab character
# (results have format: model_name\tdataset_name\tmetric_name\tmetric_value)
results = [r.replace('\n','').split('\t') for r in results if '\t' in r]

In [3]:
def remove_underscore_after(val): # Remove underscore, and keep the part after the underscore
    return val.split('_')[-1]
def remove_underscore_before(val): # Remove underscore, and keep the part before the underscore
    return val.split('_')[0]
def map2d(func, grid): # Mapping for 2d arrays, from: https://stackoverflow.com/questions/70742445/elegant-map-over-2d-list
    return [[func(value) for value in row] for row in grid]
def full_display(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(df)

df = pd.DataFrame(results, columns = ['model', 'dataset', 'metric', 'value'])
df['value'] = pd.to_numeric(df['value'])
df['model'] = df['model'].map(lambda x: '-'.join(x.split('-')[2:]))
models = df['model'].str.split('-').tolist()

# Remove all underscores from our 2d list, keep one list of the param names and one with param vals
model_names_list = map2d(remove_underscore_before, models)
model_names = np.array(model_names_list)
model_val_list = map2d(remove_underscore_after, models)
model_vals = np.array(model_val_list)

In [4]:
model_vals.shape, model_names.shape

((11470, 14), (11470, 14))

In [5]:
# Splitting model name into more columns
for i in range(model_names.shape[-1]):
    name = model_names[0][i]
    val = model_vals[:,i]
    df[name] = val
    try:
       df[name] = pd.to_numeric(df[name]) 
    except:
        pass

# Remove columns that aren't relevant
df = df.drop(['fold', 'PL', 'vit', 'model', 'data', 'ALL', 'ratio', 'kw'], axis = 1) #'method', 'AL.iter', 

# Replace 'None' with NaN, to allow conversion to numerical
df['AL.iter'] = df['AL.iter'].replace('None', np.nan)
df['AL.iter'] = pd.to_numeric(df['AL.iter'])
df['AL.epochs'] = df['AL.epochs'].replace('None', np.nan)
df['AL.epochs'] = pd.to_numeric(df['AL.epochs'])

cols = sorted(df.columns.tolist())
print(cols)
# The ratio have the format: 'ratio_xxx' where xxx is some float.
# We only want that float value, and need to convert it from string to float.
# df['ratio'] = pd.to_numeric(df['ratio'])
display(df)

# Group by all columns except the performance values, then compute mean, std and number of model runs for the performance
df_grouped = df.groupby(list(set(cols)-set(['value'])), dropna = False).agg({'value':['mean', 'std', 'count']})

['AL.epochs', 'AL.iter', 'bs', 'dataset', 'epochs', 'lr', 'method', 'metric', 'value']


,dataset,metric,value,method,AL.iter,AL.epochs,epochs,lr,bs
0,RSICD-CLS,zeroshot-val-top1,60.99,base,NaN,NaN,5,0.00005,64
1,UCM-CLS,zeroshot-val-top1,56.84,base,NaN,NaN,5,0.00005,64
2,RSICD,image_to_text_R@1,4.76,base,NaN,NaN,5,0.00005,64
3,RSICD,image_to_text_R@5,20.05,base,NaN,NaN,5,0.00005,64
4,RSICD,image_to_text_R@10,33.88,base,NaN,NaN,5,0.00005,64
...,...,...,...,...,...,...,...,...,...
11465,Sydney,image_to_text_R@5,4.63,base,3.0,1.0,1,0.00001,64
11466,Sydney,image_to_text_R@10,10.46,base,3.0,1.0,1,0.00001,64
11467,Sydney,text_to_image_R@1,0.60,base,3.0,1.0,1,0.00001,64
11468,Sydney,text_to_image_R@5,5.03,base,3.0,1.0,1,0.00001,64


In [6]:
display(df_grouped)

value  \
                                                                             mean   
metric            dataset AL.epochs method bs  epochs lr       AL.iter              
image_to_text_R@1 RSICD   1.0       base   64  1      0.000010 3.0       1.100000   
                          5.0       base   64  20     0.000010 3.0       3.570000   
                                                               5.0       3.570000   
                                                               7.0       1.830000   
                          10.0      base   64  10     0.000010 3.0       2.700000   
...                                                                           ...   
zeroshot-val-top1 UCM-CLS NaN       ours   128 30     0.000050 NaN      64.811111   
                                                      0.000500 NaN       5.856667   
                                               35     0.000005 NaN      59.240000   
                                                      0.000050 NaN      65.145556   
                                                      0.000500 NaN       5.906667   

                                                                                  \
                                                                             std   
metric            dataset AL.epochs method bs  epochs lr       AL.iter             
image_to_text_R@1 RSICD   1.0       base   64  1      0.000010 3.0           NaN   
                          5.0       base   64  20     0.000010 3.0           NaN   
                                                               5.0           NaN   
                                                               7.0           NaN   
                          10.0      base   64  10     0.000010 3.0      1.230366   
...                                                                          ...   
zeroshot-val-top1 UCM-CLS NaN       ours   128 30     0.000050 NaN      1.539297   
                                                      0.000500 NaN      2.621769   
                                               35     0.000005 NaN      2.820120   
                                                      0.000050 NaN      1.852924   
                                                      0.000500 NaN      2.690938   

                                                                              
                                                                       count  
metric            dataset AL.epochs method bs  epochs lr       AL.iter        
image_to_text_R@1 RSICD   1.0       base   64  1      0.000010 3.0         1  
                          5.0       base   64  20     0.000010 3.0         1  
                                                               5.0         1  
                                                               7.0         1  
                          10.0      base   64  10     0.000010 3.0         2  
...                                                                      ...  
zeroshot-val-top1 UCM-CLS NaN       ours   128 30     0.000050 NaN         9  
                                                      0.000500 NaN         9  
                                               35     0.000005 NaN         9  
                                                      0.000050 NaN         9  
                                                      0.000500 NaN         9  

[1560 rows x 3 columns]

In [7]:
df_grouped.to_csv('eval.csv')

In [8]:
df_results = df_grouped.reset_index()
df_results['AL.epochs'] = df_results['AL.epochs'].fillna(df_results['epochs'])
df_results = df_results.sort_values(['metric', 'dataset', ('value', 'mean')])
# df_results = df_results.groupby(['metric', 'dataset'])
# df_results = df_grouped.reset_index()
# df_results = df_results.sort_values(['metric', 'dataset', ('value', 'mean')])

# df_2 = df_2[(df_2['method'] == 'ours') & (df_2['ratio'] == 0.2) & (df_2['PL'] == 'ot.image')] # (df_2['metric'] == 'image_to_text_R@5') &
# df_2 = df_2[(df_2['epochs'] > 20)  & (df_2['bs'] == 64) & (df_2['lr']==5e-5)] # &(df_2['lr']==5e-5)
# df_2 = df_2[(df_2['lr']==5e-5) & (df_2['bs'] == 64) & (df_2['method'] == 'base')] # &(df_2['lr']==5e-5)
display(df_results)

metric  dataset AL.epochs method   bs epochs       lr  \
                                                                         
38    image_to_text_R@1    RSICD       5.0   ours   64      5  0.00050   
50    image_to_text_R@1    RSICD      25.0   ours   64     25  0.00050   
59    image_to_text_R@1    RSICD       5.0   ours  128      5  0.00050   
53    image_to_text_R@1    RSICD      30.0   ours   64     30  0.00050   
41    image_to_text_R@1    RSICD      10.0   ours   64     10  0.00050   
...                 ...      ...       ...    ...  ...    ...      ...   
1558  zeroshot-val-top1  UCM-CLS      35.0   ours  128     35  0.00005   
1514  zeroshot-val-top1  UCM-CLS      20.0   base   64     20  0.00005   
1515  zeroshot-val-top1  UCM-CLS      25.0   base   64     25  0.00005   
1517  zeroshot-val-top1  UCM-CLS      35.0   base   64     35  0.00005   
1516  zeroshot-val-top1  UCM-CLS      30.0   base   64     30  0.00005   

     AL.iter      value                  
                   mean       std count  
38       NaN   0.070000  0.060000     9  
50       NaN   0.080000  0.030000     9  
59       NaN   0.080000  0.030000     9  
53       NaN   0.090000  0.045000     9  
41       NaN   0.100000  0.030000     9  
...      ...        ...       ...   ...  
1558     NaN  65.145556  1.852924     9  
1514     NaN  66.242222  2.822781     9  
1515     NaN  66.857778  3.878353     9  
1517     NaN  67.378889  2.900217     9  
1516     NaN  67.712222  2.909960     9  

[1560 rows x 11 columns]

In [9]:
df_results.shape

(1560, 11)

In [10]:
# Results with active learning
full_display(df_results[df_results['AL.iter']>0].groupby(['metric', 'dataset']).tail(3))

metric    dataset AL.epochs method  bs epochs       lr  \
                                                                           
24     image_to_text_R@1      RSICD      15.0   base  64     15  0.00005   
6      image_to_text_R@1      RSICD      10.0   base  64     25  0.00001   
30     image_to_text_R@1      RSICD      20.0   base  64     20  0.00005   
87     image_to_text_R@1     Sydney      10.0   base  64     25  0.00001   
79     image_to_text_R@1     Sydney       5.0   base  64     20  0.00001   
81     image_to_text_R@1     Sydney       5.0   base  64     20  0.00001   
157    image_to_text_R@1        UCM       5.0   base  64     20  0.00001   
165    image_to_text_R@1        UCM      10.0   base  64     25  0.00001   
164    image_to_text_R@1        UCM      10.0   base  64     25  0.00001   
265   image_to_text_R@10      RSICD      20.0   base  64     20  0.00005   
258   image_to_text_R@10      RSICD      15.0   base  64     15  0.00005   
259   image_to_text_R@10      RSICD      15.0   base  64     15  0.00005   
317   image_to_text_R@10     Sydney      10.0   base  64     25  0.00001   
315   image_to_text_R@10     Sydney       5.0   base  64     20  0.00001   
313   image_to_text_R@10     Sydney       5.0   base  64     20  0.00001   
395   image_to_text_R@10        UCM      10.0   base  64     25  0.00001   
415   image_to_text_R@10        UCM      15.0   base  64     15  0.00005   
420   image_to_text_R@10        UCM      20.0   base  64     20  0.00005   
493    image_to_text_R@5      RSICD      15.0   base  64     15  0.00005   
492    image_to_text_R@5      RSICD      15.0   base  64     15  0.00005   
499    image_to_text_R@5      RSICD      20.0   base  64     20  0.00005   
551    image_to_text_R@5     Sydney      10.0   base  64     25  0.00001   
549    image_to_text_R@5     Sydney       5.0   base  64     20  0.00001   
547    image_to_text_R@5     Sydney       5.0   base  64     20  0.00001   
625    image_to_text_R@5        UCM       5.0   base  64     20  0.00001   
629    image_to_text_R@5        UCM      10.0   base  64     25  0.00001   
654    image_to_text_R@5        UCM      20.0   base  64     20  0.00005   
726    text_to_image_R@1      RSICD      15.0   base  64     15  0.00005   
727    text_to_image_R@1      RSICD      15.0   base  64     15  0.00005   
733    text_to_image_R@1      RSICD      20.0   base  64     20  0.00005   
783    text_to_image_R@1     Sydney       5.0   base  64     20  0.00001   
785    text_to_image_R@1     Sydney      10.0   base  64     25  0.00001   
781    text_to_image_R@1     Sydney       5.0   base  64     20  0.00001   
860    text_to_image_R@1        UCM       5.0   base  64     20  0.00001   
867    text_to_image_R@1        UCM      10.0   base  64     25  0.00001   
888    text_to_image_R@1        UCM      20.0   base  64     20  0.00005   
960   text_to_image_R@10      RSICD      15.0   base  64     15  0.00005   
961   text_to_image_R@10      RSICD      15.0   base  64     15  0.00005   
967   text_to_image_R@10      RSICD      20.0   base  64     20  0.00005   
1019  text_to_image_R@10     Sydney      10.0   base  64     25  0.00001   
1015  text_to_image_R@10     Sydney       5.0   base  64     20  0.00001   
1017  text_to_image_R@10     Sydney       5.0   base  64     20  0.00001   
1100  text_to_image_R@10        UCM      10.0   base  64     25  0.00001   
1122  text_to_image_R@10        UCM      20.0   base  64     20  0.00005   
1117  text_to_image_R@10        UCM      15.0   base  64     15  0.00005   
1194   text_to_image_R@5      RSICD      15.0   base  64     15  0.00005   
1195   text_to_image_R@5      RSICD      15.0   base  64     15  0.00005   
1201   text_to_image_R@5      RSICD      20.0   base  64     20  0.00005   
1253   text_to_image_R@5     Sydney      10.0   base  64     25  0.00001   
1249   text_to_image_R@5     Sydney       5.0   base  64     20  0.00001   
1251   text_to_image_R@5     Sydney       5.0   base  64     20  0

In [11]:
# Results for base CLIP model (filtered a bit)
full_display(df_results[(df_results['AL.iter'].isna()) & (df_results['method'] == 'base') & (df_results['lr']==5e-5)].groupby(['metric', 'dataset']).tail(3))

metric    dataset AL.epochs method  bs epochs       lr  \
                                                                           
33     image_to_text_R@1      RSICD      25.0   base  64     25  0.00005   
35     image_to_text_R@1      RSICD      35.0   base  64     35  0.00005   
34     image_to_text_R@1      RSICD      30.0   base  64     30  0.00005   
111    image_to_text_R@1     Sydney      25.0   base  64     25  0.00005   
113    image_to_text_R@1     Sydney      35.0   base  64     35  0.00005   
112    image_to_text_R@1     Sydney      30.0   base  64     30  0.00005   
188    image_to_text_R@1        UCM      20.0   base  64     20  0.00005   
191    image_to_text_R@1        UCM      35.0   base  64     35  0.00005   
190    image_to_text_R@1        UCM      30.0   base  64     30  0.00005   
268   image_to_text_R@10      RSICD      30.0   base  64     30  0.00005   
267   image_to_text_R@10      RSICD      25.0   base  64     25  0.00005   
269   image_to_text_R@10      RSICD      35.0   base  64     35  0.00005   
345   image_to_text_R@10     Sydney      25.0   base  64     25  0.00005   
346   image_to_text_R@10     Sydney      30.0   base  64     30  0.00005   
347   image_to_text_R@10     Sydney      35.0   base  64     35  0.00005   
424   image_to_text_R@10        UCM      30.0   base  64     30  0.00005   
423   image_to_text_R@10        UCM      25.0   base  64     25  0.00005   
425   image_to_text_R@10        UCM      35.0   base  64     35  0.00005   
501    image_to_text_R@5      RSICD      25.0   base  64     25  0.00005   
502    image_to_text_R@5      RSICD      30.0   base  64     30  0.00005   
503    image_to_text_R@5      RSICD      35.0   base  64     35  0.00005   
581    image_to_text_R@5     Sydney      35.0   base  64     35  0.00005   
579    image_to_text_R@5     Sydney      25.0   base  64     25  0.00005   
580    image_to_text_R@5     Sydney      30.0   base  64     30  0.00005   
656    image_to_text_R@5        UCM      20.0   base  64     20  0.00005   
658    image_to_text_R@5        UCM      30.0   base  64     30  0.00005   
659    image_to_text_R@5        UCM      35.0   base  64     35  0.00005   
735    text_to_image_R@1      RSICD      25.0   base  64     25  0.00005   
736    text_to_image_R@1      RSICD      30.0   base  64     30  0.00005   
737    text_to_image_R@1      RSICD      35.0   base  64     35  0.00005   
812    text_to_image_R@1     Sydney      20.0   base  64     20  0.00005   
814    text_to_image_R@1     Sydney      30.0   base  64     30  0.00005   
815    text_to_image_R@1     Sydney      35.0   base  64     35  0.00005   
890    text_to_image_R@1        UCM      20.0   base  64     20  0.00005   
893    text_to_image_R@1        UCM      35.0   base  64     35  0.00005   
892    text_to_image_R@1        UCM      30.0   base  64     30  0.00005   
969   text_to_image_R@10      RSICD      25.0   base  64     25  0.00005   
971   text_to_image_R@10      RSICD      35.0   base  64     35  0.00005   
970   text_to_image_R@10      RSICD      30.0   base  64     30  0.00005   
1047  text_to_image_R@10     Sydney      25.0   base  64     25  0.00005   
1049  text_to_image_R@10     Sydney      35.0   base  64     35  0.00005   
1048  text_to_image_R@10     Sydney      30.0   base  64     30  0.00005   
1125  text_to_image_R@10        UCM      25.0   base  64     25  0.00005   
1126  text_to_image_R@10        UCM      30.0   base  64     30  0.00005   
1127  text_to_image_R@10        UCM      35.0   base  64     35  0.00005   
1203   text_to_image_R@5      RSICD      25.0   base  64     25  0.00005   
1205   text_to_image_R@5      RSICD      35.0   base  64     35  0.00005   
1204   text_to_image_R@5      RSICD      30.0   base  64     30  0.00005   
1282   text_to_image_R@5     Sydney      30.0   base  64     30  0.00005   
1281   text_to_image_R@5     Sydney      25.0   base  64     25  0.00005   
1283   text_to_image_R@5     Sydney      35.0   base  64     35  0

In [12]:
# Results for S-CLIP model (filtered a bit)
full_display(df_results[(df_results['AL.iter'].isna()) & (df_results['method'] == 'ours') & (df_results['lr']==5e-5)].groupby(['metric', 'dataset']).tail(3))

metric    dataset AL.epochs method   bs epochs       lr  \
                                                                            
70     image_to_text_R@1      RSICD      25.0   ours  128     25  0.00005   
73     image_to_text_R@1      RSICD      30.0   ours  128     30  0.00005   
76     image_to_text_R@1      RSICD      35.0   ours  128     35  0.00005   
142    image_to_text_R@1     Sydney      15.0   ours  128     15  0.00005   
151    image_to_text_R@1     Sydney      30.0   ours  128     30  0.00005   
154    image_to_text_R@1     Sydney      35.0   ours  128     35  0.00005   
226    image_to_text_R@1        UCM      25.0   ours  128     25  0.00005   
229    image_to_text_R@1        UCM      30.0   ours  128     30  0.00005   
232    image_to_text_R@1        UCM      35.0   ours  128     35  0.00005   
304   image_to_text_R@10      RSICD      25.0   ours  128     25  0.00005   
310   image_to_text_R@10      RSICD      35.0   ours  128     35  0.00005   
307   image_to_text_R@10      RSICD      30.0   ours  128     30  0.00005   
382   image_to_text_R@10     Sydney      25.0   ours  128     25  0.00005   
385   image_to_text_R@10     Sydney      30.0   ours  128     30  0.00005   
388   image_to_text_R@10     Sydney      35.0   ours  128     35  0.00005   
445   image_to_text_R@10        UCM      35.0   ours   64     35  0.00005   
463   image_to_text_R@10        UCM      30.0   ours  128     30  0.00005   
466   image_to_text_R@10        UCM      35.0   ours  128     35  0.00005   
538    image_to_text_R@5      RSICD      25.0   ours  128     25  0.00005   
541    image_to_text_R@5      RSICD      30.0   ours  128     30  0.00005   
544    image_to_text_R@5      RSICD      35.0   ours  128     35  0.00005   
598    image_to_text_R@5     Sydney      30.0   ours   64     30  0.00005   
619    image_to_text_R@5     Sydney      30.0   ours  128     30  0.00005   
622    image_to_text_R@5     Sydney      35.0   ours  128     35  0.00005   
700    image_to_text_R@5        UCM      35.0   ours  128     35  0.00005   
697    image_to_text_R@5        UCM      30.0   ours  128     30  0.00005   
694    image_to_text_R@5        UCM      25.0   ours  128     25  0.00005   
775    text_to_image_R@1      RSICD      30.0   ours  128     30  0.00005   
778    text_to_image_R@1      RSICD      35.0   ours  128     35  0.00005   
757    text_to_image_R@1      RSICD      35.0   ours   64     35  0.00005   
856    text_to_image_R@1     Sydney      35.0   ours  128     35  0.00005   
853    text_to_image_R@1     Sydney      30.0   ours  128     30  0.00005   
850    text_to_image_R@1     Sydney      25.0   ours  128     25  0.00005   
928    text_to_image_R@1        UCM      25.0   ours  128     25  0.00005   
931    text_to_image_R@1        UCM      30.0   ours  128     30  0.00005   
934    text_to_image_R@1        UCM      35.0   ours  128     35  0.00005   
1006  text_to_image_R@10      RSICD      25.0   ours  128     25  0.00005   
1009  text_to_image_R@10      RSICD      30.0   ours  128     30  0.00005   
1012  text_to_image_R@10      RSICD      35.0   ours  128     35  0.00005   
1084  text_to_image_R@10     Sydney      25.0   ours  128     25  0.00005   
1087  text_to_image_R@10     Sydney      30.0   ours  128     30  0.00005   
1090  text_to_image_R@10     Sydney      35.0   ours  128     35  0.00005   
1165  text_to_image_R@10        UCM      30.0   ours  128     30  0.00005   
1147  text_to_image_R@10        UCM      35.0   ours   64     35  0.00005   
1168  text_to_image_R@10        UCM      35.0   ours  128     35  0.00005   
1240   text_to_image_R@5      RSICD      25.0   ours  128     25  0.00005   
1246   text_to_image_R@5      RSICD      35.0   ours  128     35  0.00005   
1243   text_to_image_R@5      RSICD      30.0   ours  128     30  0.00005   
1318   text_to_image_R@5     Sydney      25.0   ours  128     25  0.00005   
1321   text_to_image_R@5     Sydney      30.0   ours  128     30  0.00005   
1324   text_to